See also : https://www.perplexity.ai/search/how-can-i-use-several-clientse-4jYeihetSze0WfAVg3LZUg 



In [ ]:
# Default setup cell.
import builtins

from devtools import debug
from dotenv import load_dotenv

setattr(builtins, "debug", debug)
load_dotenv(verbose=True)
%load_ext autoreload
%autoreload 2
%reset -f
!export PYTHONPATH=":./python"

In [ ]:
from mcp import StdioServerParameters

from python.ai_core.llm import get_llm

# set_debug(True)
# set_verbose(True)

MODEL_ID = "deepseek_chatv3_deepseek"
llm = get_llm(llm_id=MODEL_ID)

In [ ]:
import os
from pathlib import Path

from python.ai_extra.mcp_tools import mcp_agent_runner

filesystem_mcp_params = StdioServerParameters(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", str(Path.cwd().parent)],
)


pubmed_mcp_params = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

arxiv_mcp_params = StdioServerParameters(
    command="uv",
    args=["tools", "run", "@smithery/arxiv-mcp-server", "--storage-path", "/tmp"],
)

timeserver_mcp_params = StdioServerParameters(
    command="npx",
    args=["-y", "@modelcontextprotocol/mcp-server-time"],
)


r = await mcp_agent_runner(llm, [filesystem_mcp_params], "list the current directory")
debug(r)